<a href="http://landlab.github.io"><img style="float: left" src="../../../landlab_header.png"></a>

# Introduction to the `ErosionDeposition` component

<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>

This tutorial introduces the `ErosionDeposition` component, which simulates erosion and deposition in a river network over long time scales.

## Theory and Overview

`ErosionDeposition` models fluvial processes using the approach described by Davy and Lague (2009). The basic goal is to calculate rates of erosion or deposition in the various branches of a river network that is embedded in a gridded landscape (see, e.g., Tucker and Hancock, 2010). The literature has a number of different approaches to this problem. Some models assume transport-limited conditions, such that erosion or deposition result from local imbalances in sediment transport capacity (see, e.g., Willgoose, 2018). Some assume detachment-limited conditions, such that any eroded sediment is entirely removed  (e.g., Howard, 1994; Whipple and Tucker, 1999; Tucker and Whipple, 2002). Still others allow for both erosion of detachment-resistant material, and re-deposition of that material. The Davy-Lague approach falls in the latter category. A brief review of the governing equations is given below; for a fuller description, see Davy and Lague (2009).

The rate of change of channel-bed elevation, $\eta$, equals a source term (representing the rate of rock uplift relative to a given baselevel), $U$, a rate of material entrainment, $E$, and a rate of deposition of sediment from the water column, $D$.

In [ ]:
from landlab.components import LinearDiffuser
from landlab.plot import imshow_grid
from landlab import RasterModelGrid
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.pyplot import figure, show, plot, xlabel, ylabel, title
import numpy as np

In [ ]:
from landlab.components import FlowAccumulator, FastscapeEroder
from landlab import load_params

In [ ]:
input_file = './coupled_params.txt'
inputs = load_params(input_file) # load the data into a dictionary

nrows = inputs['nrows']
ncols = inputs['ncols']
dx = inputs['dx']
uplift_rate = inputs['uplift_rate']
total_t = inputs['total_time']
dt = inputs['dt']

nt = int(total_t // dt) #this is how many loops we'll need
uplift_per_step = uplift_rate * dt

# illustrate what the MPD looks like:
print(inputs)

In [ ]:
mg = RasterModelGrid((nrows, ncols), dx)
z = mg.add_zeros('topographic__elevation', at='node')
# add some roughness, as this lets "natural" channel planforms arise
initial_roughness = np.random.rand(z.size) / 100000.
z += initial_roughness
for edge in (mg.nodes_at_left_edge, mg.nodes_at_right_edge):
    mg.status_at_node[edge] = mg.BC_NODE_IS_CLOSED
for edge in (mg.nodes_at_top_edge, mg.nodes_at_bottom_edge):
    mg.status_at_node[edge] = mg.BC_NODE_IS_FIXED_VALUE

In [ ]:
fr = FlowAccumulator(mg)
sp = FastscapeEroder(mg, **inputs['stream_power'])
lin_diffuse = LinearDiffuser(mg, **inputs['linear_diffuser'])

In [ ]:
for i in range(nt):
    # lin_diffuse.run_one_step(dt) no diffusion this time
    fr.run_one_step() # run_one_step isn't time sensitive, so it doesn't take dt as input
    sp.run_one_step(dt)
    mg.at_node['topographic__elevation'][mg.core_nodes] += uplift_per_step # add the uplift
    if i % 20 == 0:
        print ('Completed loop %d' % i)

In [ ]:
figure('topo without diffusion')
imshow_grid(mg, 'topographic__elevation', grid_units=['km','km'], var_name='Elevation (km)')

In [ ]:
z[:] = initial_roughness
for i in range(nt):
    lin_diffuse.run_one_step(dt)  # no diffusion this time
    fr.run_one_step() # run_one_step isn't time sensitive, so it doesn't take dt as input
    sp.run_one_step(dt)
    mg.at_node['topographic__elevation'][mg.core_nodes] += uplift_per_step # add the uplift
    if i % 20 == 0:
        print ('Completed loop %d' % i)

In [ ]:
figure('topo with diffusion')
imshow_grid(mg, 'topographic__elevation', grid_units=['km','km'],
            var_name='Elevation (km)')

In [ ]:
from landlab.components import ChannelProfiler, PrecipitationDistribution
from matplotlib.pyplot import loglog

In [ ]:
z[:] = initial_roughness

Instantiate the storm generator. This time, we're going to mix an input file for some components with manual definition of others (that we already defined above).

In [ ]:
dt = 0.1
total_t = 250.

storm_inputs = load_params('./coupled_params_storms.txt')
precip = PrecipitationDistribution(total_t=total_t, delta_t=dt, **storm_inputs)
print(storm_inputs)

In [ ]:
# make a color mapping appropriate for our time duration
norm = mpl.colors.Normalize(vmin=0, vmax=total_t)
map_color = cm.ScalarMappable(norm=norm, cmap="viridis")

Now run:

In [ ]:
out_interval = 20.
last_trunc = total_t # we use this to trigger taking an output plot
for (interval_duration, rainfall_rate) in precip.yield_storm_interstorm_duration_intensity():
    if rainfall_rate > 0.:
        # note diffusion also only happens when it's raining...
        fr.run_one_step()
        sp.run_one_step(interval_duration)
        lin_diffuse.run_one_step(interval_duration)
    z[mg.core_nodes] += uplift_rate * interval_duration
    this_trunc = precip.elapsed_time // out_interval
    if this_trunc != last_trunc:  # time to plot a new profile!
        print('made it to time %d' % (out_interval * this_trunc))
        last_trunc = this_trunc
        figure("long_profiles")
        # get and plot the longest profile
        cp = ChannelProfiler(mg)
        cp.run_one_step()
        cp.plot_profiles(color = map_color.to_rgba(precip.elapsed_time))
        
    # no need to track elapsed time, as the generator will stop automatically
# make the figure look nicer:
figure("long_profiles")
xlabel('Distance upstream (km)')
ylabel('Elevation (km)')
title('Long profiles evolving through time')
mpl.pyplot.colorbar(map_color)

Note that the "wobbles" in the long profile here are being created by the stochastic storm sequence. We could reduce their impact by reducing the storm-interstorm timescales, or allowing diffusion while it's not raining, but we've chosen not to here to show that the storms are having an effect.

In [ ]:
figure('topo with diffusion and storms')
imshow_grid(mg, 'topographic__elevation', grid_units=['km','km'], var_name='Elevation (km)')

We can also plot the location of the channels in map view. 

Here we plot all channel segments with drainage area greater than 0.01 square kilometers in the seven biggest drainage basins. 

In [ ]:
cp = ChannelProfiler(mg, number_of_watersheds=7, minimum_channel_threshold=0.01, main_channel_only=False)
cp.run_one_step()
cp.plot_profiles_in_map_view()

Next we make a slope area plot. 

In [ ]:
figure('final slope-area plot')
loglog(mg.at_node['drainage_area'], mg.at_node['topographic__steepest_slope'],'.')
xlabel('Drainage area (km**2)')
ylabel('Local slope')
title('Slope-Area plot for whole landscape')

That's a nice set of figures, eh? You can easily spend more time with pylab prettifying them if you want.

### Click here for more <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">Landlab tutorials</a>